In [1]:
import numpy as np
from numpy.random import randint
from numpy.linalg import matrix_power

# Function to calculate greatest common divisor (gcd) 
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

# Continued fraction expansion 
def continued_fraction(a, N):
    r = a / N
    frac = [0, int(r)]
    while frac[-1] != r:
        r = 1 / (r - int(r))
        frac.append(int(r))
    return frac

# Find period using continued fraction expansion
def find_period(a, N):
    frac = continued_fraction(a, N)
    if len(frac) % 2 == 0:
        frac.pop()
    return frac

# Perform quantum phase estimation
def quantum_phase_estimation(U, n):
    # Create Hadamard gate
    H = 1 / np.sqrt(2) * np.array([[1, 1], [1, -1]])

    # Initialize qubits
    qubits = np.zeros((2**n, 1))
    qubits[0] = 1

    # Apply Hadamard gate to first register
    for i in range(n):
        qubits = np.kron(H, np.eye(2**(n-1))) @ qubits

    # Apply controlled-U gates
    for _ in range(int(2**(n-1))):
        qubits = np.kron(np.eye(2), U) @ qubits

    # Apply inverse quantum Fourier transform
    qubits = np.fft.ifft(qubits, axis=0)

    # Measure and return result
    return qubits.argmax()

# Perform Shor's algorithm
def shors_algorithm(N, attempts=10):
    for _ in range(attempts):
        a = randint(2, N)
        if gcd(a, N) != 1:
            return gcd(a, N)
        else:
            frac = find_period(a, N)
            for period in frac[1:]:
                if period % 2 == 0:
                    r = int(matrix_power(a, period/2))
                    if (r + 1) % N != 0:
                        p = gcd(r + 1, N)
                        return p
    return "Failed to find factors"

ModuleNotFoundError: No module named 'numpy'

In [51]:
shorsTests = shors_algorithm(123450)
print("factors: ", shorsTests)

factors:  75
